In [15]:
%%writefile data_quality_checks.py
import configparser
import psycopg2
from sql_queries import check_1_sql, oes_state_code_check
import time
import pandas as pd

Writing data_quality_checks.py


# DATA QUALITY CHECKS

## CHECK 1 : RAW TABLES LOADED

1. Count rows in raw tables. if any of the tables has rows = 0 then fail else pass

## ChecK 2: CLEAN TABLES LOADED

1. Count rows in raw tables and then count rows in clean tables. If the count of rows don't match then fail else pass.

## Check 3: DIM TABLES loaded

1. Check count of rows in dim tables. If the count of rows is 0 faile else pass.

## Check 4: FACT TABLES loaded

1. Check count of rows in fact table. If the count of rows is 0 faile else pass.


In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

# Check 1

In [16]:
%%writefile -a data_quality_checks.py

def data_quality_raw(cur):
    raw_tables = {'stg_raw_h1b_disclosures':0,'stg_raw_oes':0,'lkp_state_codes':0,'lkp_naics_codes':0}
    
    for table in raw_tables:
        cur.execute(check_1_sql.format(table))
        result = cur.fetchall()
        raw_tables[table] = result[0][0]
        
    print(raw_tables)
    
    if 0 in raw_tables.values():
        print('raw table data quality check failed')
    else:
        print('raw table data quality check passed')

Appending to data_quality_checks.py


# Check 2

In [17]:
%%writefile -a data_quality_checks.py

def data_quality_clean(cur):
    clean_tables = {'stg_clean_h1b_disclosures':0,'stg_clean_oes':0}
    
    for table in clean_tables:
        cur.execute(check_1_sql.format(table))
        result = cur.fetchall()
        clean_tables[table] = result[0][0]
        
    print(clean_tables)
        
    if 0 in clean_tables.values():
        print('clean table data quality check failed')
    else:
        print('clean table data quality check passed')


Appending to data_quality_checks.py


# Check 3

In [18]:
%%writefile -a data_quality_checks.py

def data_quality_dim(cur):
    
    dim_tables = {'employer_dim':0,'oes_by_state_dim':0}
    
    for table in dim_tables:
        cur.execute(check_1_sql.format(table))
        result = cur.fetchall()
        dim_tables[table] = result[0][0]
        
    print(dim_tables)
    
    if 0 in dim_tables.values():
        print('General dim table data quality check failed')
    else:
        print('General dim table data quality check passed')
        
    cur.execute(oes_state_code_check)
    result = cur.fetchall()
    state_cnt = result[0][0]
    
    print('distinct state codes in oes_by_state_dim = ',state_cnt)
    
    if state_cnt != 51:
        print('OES dim table state code data quality check failed')
    else:
        print('OES dim table state code data quality check passed')


    

Appending to data_quality_checks.py


# Check 4

In [19]:
%%writefile -a data_quality_checks.py

def data_quality_fact(cur):
    
    fact_tables = {'h1b_worker_fact':0}
    
    for table in fact_tables:
        cur.execute(check_1_sql.format(table))
        result = cur.fetchall()
        fact_tables[table] = result[0][0]
        
    print(fact_tables)
    
    if 0 in fact_tables.values():
        print('General fact table data quality check failed')
    else:
        print('General fact table data quality check passed')
    

Appending to data_quality_checks.py


In [8]:
data_quality_raw(cur)

{'stg_raw_h1b_disclosures': 1279010, 'stg_raw_oes': 831161, 'lkp_state_codes': 51, 'lkp_naics_codes': 2196}
raw table data quality check passed


In [13]:
data_quality_clean(cur)

{'stg_clean_h1b_disclosures': 1279010, 'stg_clean_oes': 831161}
clean table data quality check passed


In [10]:
data_quality_dim(cur)

{'employer_dim': 141736, 'oes_by_state_dim': 69497}
General dim table data quality check passed
distinct state codes in oes_by_state_dim =  51
OES dim table state code data quality check passed


In [11]:
data_quality_fact(cur)

{'h1b_worker_fact': 1279010}
General fact table data quality check passed


In [14]:
conn.close()

In [20]:
%%writefile -a data_quality_checks.py

def main():
    
    config = configparser.ConfigParser()
    config.read('dwh.cfg')
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    data_quality_raw(cur)
    data_quality_clean(cur)
    data_quality_dim(cur)
    data_quality_fact(cur)
    
    conn.close()
    

if __name__ == "__main__":
    main()

Appending to data_quality_checks.py
